# Get Features 

For each folder


In [2]:
dir_ = '/Users/arthurcalvi/Data/species/validation/tiles'

from utils import load_folder, fit_periodic_function, get_aspect, postprocess_cloud_mask, calculate_slope_with_dates
from datetime import datetime
import rasterio 
import os 
import numpy as np 
from tqdm import tqdm

error_files = []
force = True 
for folder in tqdm(os.listdir(dir_)):
    path = os.path.join(dir_, folder)
    if os.path.isdir(path):

        if os.path.exists(os.path.join(path, 'features')) and not force:
            continue
        
        try : 
            print(f'Processing {folder}...')
            #load data
            dates = [datetime.strptime(filename.split('_')[0], '%Y-%m-%d') for filename in os.listdir(os.path.join(path, 'rgb'))]
            dates.sort() 
            rgb = load_folder(os.path.join(path, 'rgb'))
            crswir = load_folder(os.path.join(path, 'crswir'))
            qa = load_folder(os.path.join(path, 'qa'))

            dir_dem = os.path.join(path, 'dl_lc', 'dem.tif')
            dem = rasterio.open(dir_dem).read(1)
            aspect = get_aspect(dem)

            new_qa = []
            for frame in tqdm(qa):
                frame_ = np.zeros_like(frame)
                mask = (frame == 1) | (frame == 3) | (frame == 6)| (frame == 8) | (frame == 9) | (frame == 10) | (frame == 11)
                frame_[mask] = 1
                #apply postprocess cloud mask
                frame_ = postprocess_cloud_mask(frame_, 5, 25)
                new_qa.append(frame_)

            qa_mask = np.array(new_qa)
            expanded_mask = 1 - qa_mask

            #do not take into account disturbed dates 
            from utils import calculate_slope_with_dates
            K = 6
            slopes = []
            for i in tqdm(range(rgb.shape[0])):
                slope = calculate_slope_with_dates(rgb[:, 0], dates, i, K ) 
                slopes.append(slope)

            slopes = np.array(slopes)

            #apply this to a 3d array 
            zero_mask = np.zeros_like(expanded_mask[0], dtype=int)
            for i in range(12, len(dates)):
                zero_mask[ abs(slopes[i]) > 10 ] = 1
                expanded_mask[i, zero_mask.astype(bool)] = 0.001

            #compute features 
            # amplitude_map_r, phase_map_r, offset_map_r = fit_periodic_function(rgb[:, 0, :, :], expanded_mask, dates)
            # amplitude_map_crswir, phase_map_crswir, offset_map_crswir = fit_periodic_function(crswir, expanded_mask, dates)

            rcc = rgb[:, 0, :, :] / (rgb[:, 0, :, :] + rgb[:, 1, :, :] + rgb[:, 2, :, :])
            amplitude_map_rcc, phase_map_rcc, offset_map_rcc = fit_periodic_function(rcc, expanded_mask, dates)

            #write features with rasterio in 3 different .tif files in a new folder called 'features' 
            #write dem and aspect 
            folder_path = os.path.join(path, 'features')
            os.makedirs(folder_path, exist_ok=True)

            # with rasterio.open(dir_dem) as src:
            #     profile = src.profile
            #     profile.update(count=2, dtype=aspect.dtype)
            #     with rasterio.open(os.path.join(folder_path, 'elevation_aspect.tif'), 'w', **profile) as dst:
            #         dst.write(np.stack([dem, aspect], axis=0))

            # #write features of r channel 
            # with rasterio.open(dir_dem) as src:
            #     profile = src.profile
            #     profile.update(count=3, dtype=amplitude_map_r.dtype)
            #     with rasterio.open(os.path.join(folder_path, 'r_APO.tif'), 'w', **profile) as dst:
            #         dst.write(np.stack([amplitude_map_r, phase_map_r, offset_map_r], axis=0))

            # #write features of crswir channel
            # with rasterio.open(dir_dem) as src:
            #     profile = src.profile
            #     profile.update(count=3, dtype=amplitude_map_crswir.dtype)
            #     with rasterio.open(os.path.join(folder_path, 'crswir_APO.tif'), 'w', **profile) as dst:
            #         dst.write(np.stack([amplitude_map_crswir, phase_map_crswir, offset_map_crswir], axis=0))

            #write features of rcc channel
            with rasterio.open(dir_dem) as src:
                profile = src.profile
                profile.update(count=3, dtype=amplitude_map_rcc.dtype)
                with rasterio.open(os.path.join(folder_path, 'rcc_APO.tif'), 'w', **profile) as dst:
                    dst.write(np.stack([amplitude_map_rcc, phase_map_rcc, offset_map_rcc], axis=0))

        except Exception as e:
            print(f'Error processing {folder} : {e}')
            error_files.append(folder)
            continue


  0%|          | 0/80 [00:00<?, ?it/s]

Processing tile_81_20200102_20241230_Corse_occidentale_training...


100%|██████████| 47/47 [00:00<00:00, 124.41it/s]
/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1805/1195185373.py:38: UserWarning: Possible precision loss converting image of type float64 to uint8 as required by rank filters. Convert manually using skimage.util.img_as_ubyte to silence this warning.
  frame_ = postprocess_cloud_mask(frame_, 5, 25)
100%|██████████| 47/47 [00:00<00:00, 100.09it/s]
/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1805/1195185373.py:64: RuntimeWarning: divide by zero encountered in divide
  rcc = rgb[:, 0, :, :] / (rgb[:, 0, :, :] + rgb[:, 1, :, :] + rgb[:, 2, :, :])
/var/folders/1c/h8t9_vd53gsgz_wm9kswm8wm0000gp/T/ipykernel_1805/1195185373.py:64: RuntimeWarning: invalid value encountered in divide
  rcc = rgb[:, 0, :, :] / (rgb[:, 0, :, :] + rgb[:, 1, :, :] + rgb[:, 2, :, :])
  1%|▏         | 1/80 [00:11<15:10, 11.52s/it]

Processing tile_24_20200102_20241230_Massif_vosgien_central_training...


  2%|▎         | 2/80 [00:22<14:29, 11.15s/it]

...

 98%|█████████▊| 78/80 [12:46<00:19,  9.74s/it]

Processing tile_11_20200102_20241230_Côtes_et_plateaux_de_la_Manche_training...


 99%|█████████▉| 79/80 [12:57<00:09,  9.97s/it]

Processing tile_35_20200102_20241230_Premier_plateau_du_Jura_training...


100%|██████████| 80/80 [13:03<00:00,  9.79s/it]


In [8]:
error_files

["tile_51_20200102_20241230_Châtaigneraie_du_Centre_et_de_l'Ouest_training",
 'tile_61_20200102_20241230_Préalpes_du_Nord_validation']